# Importando Bibliotecas

In [2]:
import faceDetection
import re
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import time

#https://www.geeksforgeeks.org/face-recognition-with-local-binary-patterns-lbps-and-opencv/

# Funcoes

In [3]:
def save_labels(labels):
    import json
    # Specify the file path
    file_path = 'labels_ids.json'

    # Save dictionary to a file
    with open(file_path, 'w') as file:
        json.dump(labels, file)


def load_labels():

    import json

    # Specify the file path
    file_path = 'labels_ids.json'

    # Load dictionary from file
    with open(file_path, 'r') as file:
        loaded_dict = json.load(file)

    return loaded_dict

In [4]:
label=load_labels()
label

{'Pedro': 1, 'Carl': 2, 'Jessica': 3, 'Amanda': 4, 'Albert Mazuz': 5}

# Treinando o Modelos

In [17]:

def train_model(label):
    # Create lists to store the face samples and their corresponding labels
    faces = []
    labels = []
    
    dir_path='files/caltech_faces/train'
    # Load the images from the 'Faces' folder
    for file_name in os.listdir(dir_path):
        if file_name.endswith('.jpg'):
            # Extract the label (person's name) from the file name
            name = re.split(r'\s+\(.*\)', file_name)[0]
            print(file_name)
            # Read the image
            image = cv2.imread(os.path.join(dir_path, file_name))

            # Crop the detected face region
            cropped_faces = faceDetection.alinha_face(image,return_face=True)
            # Se existir apenas uma face, salva a imagem e o label
            if len(cropped_faces)==1:
                # Pega as informações da imagem na lista
                face_crop=cropped_faces[0]
                # Pega a imagem no dicionario
                face_crop=face_crop['image']
                # Transforma na escala de cinza
                face_crop=cv2.cvtColor(face_crop, cv2.COLOR_BGR2GRAY)
                # Append the face sample and label to the lists
                faces.append(face_crop)
                labels.append(label[name])
            else:
                print('Imagem com mais de uma face. Rejeitada')



 
    # Train the face recognition model using the faces and labels
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    # recognizer = cv2.face.LBPHFaceRecognizer_create(radius=2, neighbors=16, grid_x=8, grid_y=8)
    start=time.time()
    recognizer.train(faces, np.array(labels))
    
    # Save the trained model to a file
    recognizer.save('id_recognition_model.xml')
    return recognizer
 
recognizer = cv2.face.LBPHFaceRecognizer_create()
# Train the model
Recognizer =train_model(label)
Recognizer


Albert Mazuz (0).jpg
(480, 640, 3)
Albert Mazuz (1).jpg
(480, 640, 3)
Albert Mazuz (10).jpg
(480, 640, 3)
Albert Mazuz (11).jpg
(480, 640, 3)
Albert Mazuz (12).jpg
(480, 640, 3)
Albert Mazuz (13).jpg
(480, 640, 3)
Albert Mazuz (14).jpg
(480, 640, 3)
Albert Mazuz (15).jpg
(480, 640, 3)
Albert Mazuz (16).jpg
(480, 640, 3)
Albert Mazuz (17).jpg
(480, 640, 3)
Albert Mazuz (18).jpg
(480, 640, 3)
Albert Mazuz (19).jpg
(480, 640, 3)
Albert Mazuz (2).jpg
(480, 640, 3)
Albert Mazuz (20).jpg
(480, 640, 3)
Albert Mazuz (21).jpg
(480, 640, 3)
Albert Mazuz (22).jpg
(480, 640, 3)
Albert Mazuz (23).jpg
(480, 640, 3)
Albert Mazuz (3).jpg
(480, 640, 3)
Albert Mazuz (4).jpg
(480, 640, 3)
Albert Mazuz (5).jpg
(480, 640, 3)
Albert Mazuz (6).jpg
(480, 640, 3)
Albert Mazuz (7).jpg
(480, 640, 3)
Albert Mazuz (8).jpg
(480, 640, 3)
Albert Mazuz (9).jpg
(480, 640, 3)
Amanda (1).jpg
(592, 896, 3)
Amanda (10).jpg
(592, 896, 3)
Amanda (11).jpg
(592, 896, 3)
Amanda (12).jpg
(592, 896, 3)
Amanda (13).jpg
(592, 896, 

< cv2.face.LBPHFaceRecognizer 00000207C42B7AD0>

# Reconhecendo Usuarios

In [18]:
Recognizer = cv2.face.LBPHFaceRecognizer_create()
Recognizer.read('id_recognition_model.xml')



def recognize_face(recognizer,label,imagem):
    """Recognize a cropped image"""

    # Reverse keys and values in the dictionary
    label_name = {value: key for key, value in label.items()}
    #Preparar Imagem
    imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)
    label,confidence=recognizer.predict(imagem)
    return {'name':label_name[label],'confidence':confidence}

dir_path='files/caltech_faces/test'

for file_name in os.listdir(dir_path):
     if file_name.endswith('.jpg'):
            image_path=os.path.join(dir_path, file_name)
            # Extract the label (person's name) from the file name
            name = re.split(r'\s+\(.*\)', file_name)[0]
            imagem = cv2.imread(image_path)
            # Se número de faces =1
            if faceDetection.numero_de_faces(imagem)==1:    
                #Crop the image
                faces_crop = faceDetection.alinha_face(imagem,return_face=True)
                if len(faces_crop)==1:
                    face_crop=faces_crop[0]['image']
                    #Recognize the person

                    recognition=recognize_face(Recognizer,label,face_crop)
                    name_recognized=recognition['name']
                    confidence=recognition['confidence']
                    print(f'Original name x Recognized Name')
                    print(f'{name} - {name_recognized}')
                    print(f'Confidence= {confidence}')

(480, 640, 3)
Original name x Recognized Name
Albert Mazuz - Albert Mazuz
Confidence= 46.82630539815293
(800, 800, 3)
Original name x Recognized Name
Albert Mazuz.jpg - Carl
Confidence= 51.05106402358276
(592, 896, 3)
Original name x Recognized Name
Amanda - Amanda
Confidence= 25.362745747691907
(592, 896, 3)
Original name x Recognized Name
Carl - Carl
Confidence= 20.866773081919167
(592, 896, 3)
Original name x Recognized Name
Jessica - Pedro
Confidence= 26.814271081638175
(592, 896, 3)
Original name x Recognized Name
Pedro - Pedro
Confidence= 22.597493331669007


# Capturando Imagens

In [5]:


# Create a face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
 
# Generate a face recognition model
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('id_recognition_model.xml')

# Function to capture images and store in dataset folder
def capture_images(User):
    path='files/caltech_faces/train'
    # Create a directory to store the captured images
    if not os.path.exists(path):
        os.makedirs('files/caltech_faces')
 
    # Open the camera
    cap = cv2.VideoCapture(0)
 
    # Set the image counter as 0
    count = 0
 
    while True:
        # Read a frame from the camera
        ret, frame = cap.read()
 
        # # Store the captured face images in the Faces folder
        # cv2.imwrite(f'{path}/{User} ({count}).jpg', frame)
        # Convert the frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        img=frame
 
        # Detect faces in the grayscale frame
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
 
        # Draw rectangles around the faces and store the images
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            
            # recognize_face(recognizer,label,imagem)
 

 
            count += 1
 
        # Display the frame with face detection
        cv2.imshow('Capture Faces', frame)
 
        # Break the loop if the 'q' key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
 
        # Break the loop after capturing a certain number of images
        if count >=24:
            break
 
    # Release the camera and close windows
    cap.release()
    cv2.destroyAllWindows()

capture_images('Albert Mazuz')

# Adicionando Usuarios

In [20]:
from tqdm import tqdm
label=load_labels()
new_user_name='Albert Mazuz'
if new_user_name not in label:
    last_label_number=max(labels.values())
    label[new_user_name]=last_label_number+1
    save_labels(label)

label

# capture_images(new_user_name)

# update_model(new_user_name)
    # Create lists to store the face samples and their corresponding labels
faces = []
labels = []

dir_path='files/caltech_faces/train'
# Load the images from the 'Faces' folder
for file_name in tqdm(os.listdir(dir_path)):
    if file_name.endswith('.jpg'):
        # Extract the label (person's name) from the file name
        name = re.split(r'\s+\(.*\)', file_name)[0]
      
        if name==new_user_name:
            # Read the image
            image = cv2.imread(os.path.join(dir_path, file_name))

            # print(image.shape)
            # Crop the detected face region
            cropped_faces = faceDetection.alinha_face(image,return_face=True)
            # Se existir apenas uma face, salva a imagem e o label
            if len(cropped_faces)==1:
                # Pega as informações da imagem na lista
                face_crop=cropped_faces[0]
                # Pega a imagem no dicionario
                face_crop=face_crop['image']
                # Transforma na escala de cinza
                face_crop=cv2.cvtColor(face_crop, cv2.COLOR_BGR2GRAY)
                # Append the face sample and label to the lists
                faces.append(face_crop)
                labels.append(label[name])
            else:
                print('Imagem com mais de uma face. Rejeitada') 

recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('id_recognition_model.xml')
recognizer.update(faces,np.array(labels))



  0%|          | 0/102 [00:00<?, ?it/s]

(480, 640, 3)


  1%|          | 1/102 [00:02<04:07,  2.45s/it]

Imagem com mais de uma face. Rejeitada
(480, 640, 3)


  2%|▏         | 2/102 [00:04<03:38,  2.19s/it]

...

 22%|██▏       | 22/102 [00:40<02:27,  1.84s/it]

(480, 640, 3)


 23%|██▎       | 23/102 [00:42<02:22,  1.80s/it]

(480, 640, 3)


100%|██████████| 102/102 [00:44<00:00,  2.32it/s]
